In [3]:
import os
import torch 
import json
import numpy as np 
from torch.utils.data import DataLoader
from tqdm import tqdm 

from model.bert.bertconfig import BertConfig
from model.fusemodel import DesignFuseModel, FuseModel2TasksNewDropout

image_dropout = 0.3
vocab_dict_file = 'data/new_data/vocab/vocab_dict.json'
vocab_file = 'data/new_data/vocab/vocab.txt'
attr_dict_file = 'data/new_data/equal_processed_data/dict/attr_relation_dict.json'

split_layers = 0
fuse_layers = 3
n_img_expand = 6

# model
split_config = BertConfig(num_hidden_layers=split_layers)
fuse_config = BertConfig(num_hidden_layers=fuse_layers, image_dropout=image_dropout)
model = DesignFuseModel(split_config, fuse_config, vocab_file, n_img_expand=n_img_expand, word_match=True)


In [4]:
sum(param.numel() for param in model.parameters())

31080194

In [6]:
52343810 * 3

157031430

In [1]:
from ltp import LTP
ltp = LTP('data/pretrained_model/ltp_base/') # 默认加载 Small 模型


RuntimeError: CUDA out of memory. Tried to allocate 62.00 MiB (GPU 0; 23.70 GiB total capacity; 534.54 MiB already allocated; 46.56 MiB free; 568.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
sum(param.numel() for param in ltp.model.parameters())

137875216

In [4]:
500000000 - 137875216

362124784

In [9]:
size = 500000000*32/8/1024/1024/1024
f"{size:.3f} GB"

'1.863 GB'

In [1]:
title_model = 52343041
title_model*3

157029123

In [15]:
362124784-title_model*5

100409579

In [12]:
52343041/4

13085760.25

In [2]:
l = [1,2,3]
for i in l:
    if i == 2:
        del i

In [9]:
a = torch.tensor(1.)
b = torch.tensor(4.)
torch.tensor([a, b])

tensor([1., 4.])

In [3]:
import json
from tqdm import tqdm 
val_file = 'data/new_data/divided/title/shuffle/fine5000_0.25posaug.txt'

In [5]:
with open(val_file, 'r') as f:
    for line in tqdm(f):
        pass

10848it [00:00, 31086.15it/s]


In [4]:
import os
import random
import torch 
import json
import numpy as np 
from tqdm import tqdm 
import copy 

seed = 0
random.seed(seed)
np.random.seed(seed)


val_file = 'data/new_data/divided/attr/fine5000.txt'
vocab_dict_file = 'data/new_data/vocab/vocab_dict.json'
vocab_file = 'data/new_data/vocab/vocab.txt'
relation_dict_file = 'data/new_data/equal_processed_data/attr_relation_dict.json'
save_file = 'data/new_data/divided/attr/val/fine5000.txt'
with open(relation_dict_file, 'r') as f:
    relation_dict = json.load(f)

In [5]:

rets = []
with open(val_file, 'r') as f:
    for line in tqdm(f):
        item = json.loads(line)
        if item['key_attr']: # 必须有属性
            for query, attr in item['key_attr'].items():
                key_attr = {}
                match = {}
                new_item = copy.deepcopy(item)
                if random.random() < 0.5: # 替换，随机挑选一个词替换
                    label = 0
                    attr_list = random.sample(relation_dict[attr]['similar_attr'], 1)[0]
                    if len(attr_list) == 1:
                        split = attr_list
                    else:
                        attr = random.sample(attr_list, 1)[0]
                        split = [attr]
                else: 
                    label = 1
                    split = [attr]
                    
                key_attr[query] = split[0]
                match[query] = label
                new_item['key_attr'] = key_attr
                new_item['match'] = match
                rets.append(json.dumps(new_item, ensure_ascii=False)+'\n')
        
with open(save_file, 'w') as f:
    f.writelines(rets)

5000it [00:18, 277.28it/s]


In [1]:
import numpy as np 
import random

In [1]:
a = 0.121324334
round(a, 5)

0.12132

In [1]:
import torch

In [8]:
a = torch.tensor([[1.,2],[3,4]])

In [9]:
torch.softmax(a, dim=-1)

tensor([[0.2689, 0.7311],
        [0.2689, 0.7311]])

In [63]:
l = np.array([[1,2],[3],[4]], dtype='object')

In [4]:
l = [[1,2]]

In [5]:
random.sample(l, 1)

[[1, 2]]

In [64]:
np.random.choice(l)

ValueError: a must be 1-dimensional

In [31]:
p = 0.5
num_list = [205+83,317+723,100]

N = len(num_list)
S = sum(num_list)
copy_list = num_list.copy()
for i in range(N):
    n = copy_list[i]
    pos = n * (1-p)
    neg = (S-n) * p / (N-1)
    num_list[i] = int(pos + neg)
num_list

[429, 617, 382]

In [1]:
[40+68,43+144,131]

[108, 187, 131]

In [4]:
import json
attr_dict_file = 'data/new_data/equal_processed_data/attr_to_attrvals.json'
vocab_dict_file = 'data/new_data/vocab/vocab_dict.json'
with open(attr_dict_file, 'r') as f:
    attr_dict = json.load(f)
with open(vocab_dict_file, 'r') as f:
    vocab_dict = json.load(f)

In [6]:
total = 0
num_attr = 0
for query, attr_list in attr_dict.items():
    for attr in attr_list:
        num_attr += vocab_dict[attr]
for word, n in vocab_dict.items():
    total += n

In [7]:
num_attr

302501

In [8]:
total

995299

In [1]:
import json 

attr_dict_file = 'data/new_data/equal_processed_data/attr_to_attrvals.json'
vocab_dict_file = 'data/new_data/vocab/vocab_dict.json'
with open(attr_dict_file, 'r') as f:
    attr_dict = json.load(f)
with open(vocab_dict_file, 'r') as f:
    vocab_dict = json.load(f)
def get_negative_dict(attr_dict, vocab_dict):
    proba_negative_dict = {}
    for query, attr_list in attr_dict.items():
        proba_negative_dict[query] = {}
        proba_negative_dict[query]['attr_list'] = attr_list
        proba_list = []
        for attr in attr_list:
            proba_list.append(vocab_dict[attr])
        proba_negative_dict[query]['attr_freq'] = proba_list
            
    return proba_negative_dict

proba_negative_dict = get_negative_dict(attr_dict, vocab_dict)

In [3]:
attr_save_file = 'data/new_data/equal_processed_data/attr_analysis.json'
with open(attr_save_file, 'w') as f:
    json.dump(proba_negative_dict, f, ensure_ascii=False, indent=4)

In [1]:
import os
from tqdm import tqdm 
import json
fine_path = 'data/new_data/split_word/coarse10412.txt'
fine_val_path = 'data/new_data/split_word/coarse10412_2.txt'
train_rets = []
val_rets = []

with open(fine_path, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        if len(train_rets) < 89588:      
            train_rets.append(json.dumps(data, ensure_ascii=False)+'\n')
        else:
            val_rets.append(json.dumps(data, ensure_ascii=False)+'\n')
print(len(train_rets))
print(len(val_rets))

# with open(fine_train_path, 'w') as f:
#     f.writelines(train_rets)
with open(fine_val_path, 'w') as f:
    f.writelines(val_rets)

100000it [01:52, 889.18it/s]


89588
10412


In [ ]:
file = 'data/equal_processed_data/nofeat/test10000_analysis.txt'


proba_negative_dict = {}
for query, attr_list in attr_dict.items():
    proba_negative_dict[query] = {}
    proba_negative_dict[query]['attr_list'] = attr_list
    proba_list = []
    for attr in attr_list:
        proba_list.append(0)
    proba_negative_dict[query]['attr_freq'] = proba_list


query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        new_key_attr = data['new_key_attr']

        for query, attr in key_attr.items():
            attr_list = proba_negative_dict[query]['attr_list']
            idx = attr_list.index(attr)
            proba_negative_dict[query]['attr_freq'][idx] += 1

In [1]:
import numpy as np

In [1]:
30 + 1452 * 0.9373 + 299 * 0.9565 + 331 * 0.9365

1986.9346

In [2]:
衣服 1452个 0.9373
裤子 299 0.9565 
鞋 331 0.9365 
包 30 1.0

0.9408143939393939

In [1]:
from tqdm import tqdm 
import json
import itertools
import random 

# 加载新的属性字典
attr_file = 'data/equal_processed_data/attr_to_attrvals.json'
with open(attr_file, 'r') as f:
    attr_dict = json.load(f)

In [2]:
file = 'data/equal_processed_data/nofeat/test10000.txt'
save_file = 'data/equal_processed_data/nofeat/test10000_analysis.txt'

rets = []
query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        title = data['title']
        key_attr = {}
        
        # 属性提取
        for query in query_list:
            attr_list = attr_dict[query]
            for attr in attr_list:
                if attr in title:
                    key_attr[query] = attr
                        
        data['new_key_attr'] = key_attr
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')
        
with open(save_file, 'w') as f:
    f.writelines(rets)

10000it [00:00, 64337.51it/s]


In [7]:
file = 'pred_title_attr_set1_B.txt'

preds = []
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        preds.append(data)
            

10000it [00:00, 83007.69it/s]


In [74]:
file = 'data/equal_processed_data/nofeat/test10000_analysis.txt'

count_dict = {}
pos_count_dict = {}
count_query = '领型'

query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        new_key_attr = data['new_key_attr']
        
        match = preds[i]['match']
        
        if count_query in new_key_attr.keys():
            attr = new_key_attr[count_query]
            if attr not in count_dict:
                count_dict[attr] = 1
            else:
                count_dict[attr] += 1
            if count_query in match:
                if match[count_query] == 1:
                    if attr not in pos_count_dict:
                        pos_count_dict[attr] = 1
                    else:
                        pos_count_dict[attr] += 1
        

10000it [00:00, 176435.87it/s]


In [3]:
l = [12221, 13685, 11393, 26, 4175, 70, 46, 177, 407, 3525, 908, 12853, 13, 11]
N = len(l)

In [14]:
l = [6122,34906,418,747]
N = len(l)

In [19]:
p = 0.4
new_l = []
for i in range(N):
    n = l[i]
    pos = n * (1-p)
    neg = 0
    for j in range(N):
        if j != i:
            neg += l[j] * p / (N-1)
    new_l.append(int(pos+neg))

In [20]:
new_l

[8482, 21915, 5820, 5974]

In [75]:
print(count_dict)
print(pos_count_dict)

{'圆领': 659, '连帽': 715, '亨利领': 113, 'V领': 284, 'U型领': 134, '高领': 647, '棒球领': 161, '一字领': 125, '西装领': 319, '翻领': 681, '斜领': 111, '双层领': 137, '堆堆领': 134, '围巾领': 122}
{'圆领': 576, '连帽': 629, 'V领': 176, '高领': 551, '西装领': 201, '翻领': 590, 'U型领': 7, '棒球领': 41, '堆堆领': 15, '围巾领': 9}


In [76]:
file = 'data/equal_processed_data/nofeat/fine45000.txt'

count_dict = {}

count_query = '领型'

query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']

        if count_query in key_attr.keys():
            attr = key_attr[count_query]
            if attr not in count_dict:
                count_dict[attr] = 1
            else:
                count_dict[attr] += 1

45000it [00:00, 179374.76it/s]


In [5]:
num_list = [6122,34906,418,747]
p = 0.9296

In [6]:
S = sum(num_list)
N = len(num_list)
for n in num_list:
    n_pos = n*(1-p)
    print(n, n_pos)

6122 430.9888000000001
34906 2457.3824000000004
418 29.427200000000006
747 52.58880000000001


In [77]:
count_dict

{'高领': 3684,
 '翻领': 3900,
 '连帽': 4541,
 'V领': 1209,
 '圆领': 4331,
 '西装领': 1366,
 '围巾领': 77,
 '棒球领': 307,
 'U型领': 33,
 '堆堆领': 161,
 '一字领': 23,
 '亨利领': 6,
 '斜领': 6,
 '双层领': 12}

In [96]:
590 / 681

0.8663729809104258

In [97]:
2535 / 2959

0.8567083474146672

In [93]:
# l = [280, 707, 344, 350]
# l = [993, 4691]
l = list(count_dict.values())
p = 0.35 # 负例概率
for x in l:
    y = x * (1-p) + (sum(l) - x) * p / (len(l) - 1)
    print(y)

2824.6153846153848
2959.2
3358.5923076923077
1282.5
3227.746153846154
1380.3230769230768
577.176923076923
720.4846153846154
549.7615384615384
629.5153846153846
543.5307692307692
532.9384615384615
532.9384615384615
536.676923076923


In [94]:
for x in l:
    y = x * (1-p)
    print(y)

2394.6
2535.0
2951.65
785.85
2815.15
887.9
50.050000000000004
199.55
21.45
104.65
14.950000000000001
3.9000000000000004
3.9000000000000004
7.800000000000001


In [70]:
1876 / 2471

0.7592067988668555

In [71]:
471 / 611

0.7708674304418985

In [70]:
file = 'data/equal_processed_data/nofeat/test10000_analysis.txt'

attr_count = 0
count = 0
pos_count = 0
query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        new_key_attr = data['new_key_attr']
        
        # if '类别' in key_attr.keys():
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
        
        # if any(q in key_attr.keys() for q in ['领型', '袖长', '衣长', '版型', '裙长', '穿着方式']):
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
        
        # if any(q in key_attr.keys() for q in ['裤型', '裤长', '裤门襟']):
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     # print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
        
        # if any(q in key_attr.keys() for q in ['闭合方式', '鞋帮高度']):
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     # print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
                
        # if key_attr == new_key_attr:
        #     if len(key_attr) == 1:
        #         count += 1
        #         if preds[i]['match']['图文'] == 1:
        #             pos_count += 1
        #         print(img_name)
        #         print(preds[i]['match'])
        #         print(key_attr)
        #         print(new_key_attr)
        #         print()
        
        if '类别' not in key_attr.keys(): # 去除包
            if preds[i]['match']['图文'] == 0: # 负例
                if key_attr == new_key_attr: # 去除隐藏属性的样本
                    if len(key_attr) >= 2: # 属性不少于两个
                        flag_1 = 0
                        flag_0 = 0
                        for key, value in preds[i]['match'].items():
                            if key != '图文':
                                if value == 1:
                                    flag_1 = 1
                                else:
                                    flag_0 = 1
                        if flag_0 and flag_1:
                            count += 1
                    
                    
            
            
        # if key_attr != new_key_attr:
        #     # count += 1
        #     # if preds[i]['match']['图文'] == 1:
        #     #     pos_count += 1
        #     for key, value in preds[i]['match'].items():
        #         if key != '图文':
        #             attr_count += 1
        #             if value == 1:
        #                 pos_count += 1
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(title)
        #     print(key_attr)
        #     print(new_key_attr)
        #     print()
        
        # if preds[i]['match']['图文'] == 0:
        #     if '类别' not in key_attr.keys():
        #         if key_attr == new_key_attr:
        #             count += 1
        #             for key, value in preds[i]['match'].items():
        #                 if key != '图文':
        #                     attr_count += 1
        #                     if value == 1:
        #                         pos_count += 1
                
        
        # if preds[i]['match']['图文'] == 1:
        #     count += 1
            
        if i > 10000:
            break
print(count)
print(pos_count)
print(attr_count)

10000it [00:00, 166552.99it/s]

2409
0
0


In [66]:
10000 - 363 - (451 + 2000 + 410)

6776

In [69]:
1000/ 6776

0.14757969303423848

In [18]:
file = 'pred_title_attr_set1_B.txt'

count = 0
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        if data['match']['图文'] == 1:
            count += 1
            
print(count)

10000it [00:00, 326661.73it/s]

3100


In [17]:
1224 / 4000

0.306

In [9]:
file = 'data/equal_processed_data/nofeat/fine5000.txt'
save_file = 'data/equal_processed_data/nofeat/mytest5000.txt'

rets = []
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        match = data['match']
        
        if '类别' in key_attr.keys():
            if random.random() < 1/3:
                match['类别'] = 0
                match['图文'] = 0
                
        else:
            flag = 1
            for key, value in key_attr.items():
                if random.random() < 0.62:
                    match[key] = 0
                    flag = 0
                if flag == 0:
                    match['图文'] = 0
                
        rets.append(data)

5000it [00:00, 154794.21it/s]


In [13]:
count = 0
pos_count = 0


for data in rets:
    img_name = data['img_name']
    title = data['title']
    key_attr = data['key_attr']
    match = data['match']
    if match['图文'] == 1:
        pos_count += 1
        
print(pos_count)

893


In [94]:
import random 
random.seed(0)

attr_dict_file = 'data/equal_processed_data/attr_to_attrvals.json'
with open(attr_dict_file, 'r') as f:
    attr_dict = json.load(f)

def get_negative_dict(attr_dict):
        negative_dict = {}
        for query, attr_list in attr_dict.items():
            negative_dict[query] = {}
            for attr in attr_list:
                l = attr_list.copy()
                l.remove(attr)
                negative_dict[query][attr] = l
        return negative_dict
negative_dict = get_negative_dict(attr_dict)


file = 'data/equal_processed_data/nofeat/fine5000.txt'
save_file = 'data/equal_processed_data/nofeat/mytest5000_nohide.txt'

rets = []
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        match = data['match']
        
        # 包 永远只有一个属性，1/3的概率为负
        if '类别' in key_attr.keys():
            if random.random() < 1/3:
                query = '类别'
                attr = key_attr[query]
                new_attr = random.sample(negative_dict[query][attr], 1)[0]
                key_attr[query] = new_attr
                title = title.replace(attr, new_attr)
                match['类别'] = 0
                match['图文'] = 0
        # 其他样本，29.2%的概率为正样本
        else:
            if random.random() < (1 - 0.292): # 以此概率为负
                match['图文'] = 0
                # 先随机选取一个属性为负
                query = random.sample(list(key_attr.keys()), 1)[0]
                attr = key_attr[query]
                new_attr = random.sample(negative_dict[query][attr], 1)[0]
                key_attr[query] = new_attr
                title = title.replace(attr, new_attr)
                match[query] = 0
                # 其余属性以一定概率变负
                for query, attr in key_attr.items():
                    if match[query] != 0:
                        if random.random() < 0.434:
                            new_attr = random.sample(negative_dict[query][attr], 1)[0]
                            key_attr[query] = new_attr
                            title = title.replace(attr, new_attr)
                            match[query] = 0
                # 隐藏属性
                # if len(key_attr) >= 2: # 属性不少于两个
                #     flag_0 = 0
                #     flag_1 = 0
                #     for query, value in match.items(): # 有正有负
                #         if query != '图文':
                #             if value == 1:
                #                 flag_1 = 1
                #             else:
                #                 flag_0 = 1
                #     if flag_0 and flag_1:
                #         if random.random() < 0.3: # 一定概率进行隐藏
                #             match_copy = match.copy()
                #             for query, value in match_copy.items():
                #                 if query != '图文':
                #                     if value == 0:
                #                         del match[query]
                #                         del key_attr[query]
            
        data['title'] = title 
        data['key_attr'] = key_attr 
        data['match'] = match 
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')

print(len(rets))
with open(save_file, 'w') as f:
    f.writelines(rets)

5000it [00:00, 54004.52it/s]

5000


In [106]:
file = 'data/equal_processed_data/mytest5000_hide.txt'

attr_count = 0
count = 0
pos_count = 0
query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        match = data['match']
        for query, attr in key_attr.items():
            attr_count += 1
            
print(attr_count)

5000it [00:01, 3674.92it/s]

10932


In [108]:
file = 'data/equal_processed_data/mytest5000_hide.txt'

attr_count = 0
count = 0
pos_count = 0
query_list = list(attr_dict.keys())
with open(file, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        img_name = data['img_name']
        title = data['title']
        key_attr = data['key_attr']
        match = data['match']
        
        # if '类别' in key_attr.keys():
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
        
        # if any(q in key_attr.keys() for q in ['领型', '袖长', '衣长', '版型', '裙长', '穿着方式']):
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
        
        # if any(q in key_attr.keys() for q in ['裤型', '裤长', '裤门襟']):
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     # print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
        
        # if any(q in key_attr.keys() for q in ['闭合方式', '鞋帮高度']):
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(key_attr)
        #     # print(new_key_attr)
        #     print()
        #     count += 1
        #     if preds[i]['match']['图文'] == 1:
        #         pos_count += 1
                
        # if key_attr == new_key_attr:
        #     if len(key_attr) == 1:
        #         count += 1
        #         if preds[i]['match']['图文'] == 1:
        #             pos_count += 1
        #         print(img_name)
        #         print(preds[i]['match'])
        #         print(key_attr)
        #         print(new_key_attr)
        #         print()
        
        if '类别' not in key_attr.keys(): # 去除包
            if match['图文'] == 0: # 负例
                if len(key_attr) >= 2: # 属性不少于两个
                    flag_1 = 0
                    flag_0 = 0
                    for key, value in match.items():
                        if key != '图文':
                            if value == 1:
                                flag_1 = 1
                            else:
                                flag_0 = 1
                    if flag_0 and flag_1:
                        count += 1
                    
                    
            
            
        # if key_attr != new_key_attr:
        #     # count += 1
        #     # if preds[i]['match']['图文'] == 1:
        #     #     pos_count += 1
        #     for key, value in preds[i]['match'].items():
        #         if key != '图文':
        #             attr_count += 1
        #             if value == 1:
        #                 pos_count += 1
        #     print(img_name)
        #     print(preds[i]['match'])
        #     print(title)
        #     print(key_attr)
        #     print(new_key_attr)
        #     print()
        
        # if preds[i]['match']['图文'] == 0:
        #     if '类别' not in key_attr.keys():
        #         if key_attr == new_key_attr:
        #             count += 1
        #             for key, value in preds[i]['match'].items():
        #                 if key != '图文':
        #                     attr_count += 1
        #                     if value == 1:
        #                         pos_count += 1
                
        
        # if preds[i]['match']['图文'] == 1:
        #     count += 1
            
        if i > 10000:
            break
print(count)
print(pos_count)
print(attr_count)

5000it [00:01, 3687.88it/s]

1899
0
0


In [86]:
1919 * 2 * 0.29

1113.02

In [6]:
import json
from tqdm import tqdm
result_file = 'output/fusion/predict/attr_0.txt'

c = 0
a = 0
with open(result_file, "r") as f:
    for line in tqdm(f):
        item = json.loads(line)
        for key, value in item["pred"].items():
            if key!=["图文"]:
                if int(value>0.5) == int(item["match"][key]>0.5):
                    c+=1
            a+=1
c/a, c, a

5000it [00:00, 103325.80it/s]


(0.7085820400583632, 10684, 15078)

In [5]:
import json
from tqdm import tqdm
result_file = 'output/fusion/predict/attr_0.txt'

c = 0
a = 0
with open(result_file, "r") as f:
    for line in tqdm(f):
        item = json.loads(line)
        # print(item)
        for key, value in item["pred"].items():
            # print(key, value)
            if key!="图文":
                if int(value>0.5) == int(item["match"][key]>0.5):
                    c+=1
                a+=1
            # else:
            #     print("1")
        # break
c/a, c, a

5000it [00:00, 112435.17it/s]


(0.9360984322286168, 9434, 10078)